## Test Half-Light Radii in spec-phot catalog
- Previous HLR from SExtractor "flux_radius" were too large for deblended objects
- Sanity + visual check spec-phot "fwhm" measure of HLR
- Does FWHM of 2D {Gaussian, Sersic} (what profile is assumed in SExtractor?) contain half the light?

In [1]:
import sys
sys.path.insert(0,'/data/emiln/XLSSC122_GalPops/Analysis/Modules/GalfitPyWrap')
import galfitwrap as gf
import os

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import glob
from scipy import ndimage
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.io import ascii
from astropy import wcs
from astropy.table import Table, hstack, join
from __future__ import division
import fnmatch
import multiprocessing as mp
%matplotlib inline

# Make plots readable
# Update plot params
# Make plots readable
params = {'legend.fontsize': 18,
	  'figure.figsize': (6,5),
	 'axes.labelsize': 22,
	 'axes.titlesize':22,
	 'xtick.labelsize':18,
	 'ytick.labelsize':18}
plt.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
plt.rc('text', usetex=True)
plt.rcParams.update(params)
# plt.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
# plt.rc('text', usetex=True)
# plt.rc('text.latex', preamble=r'\usepackage{cmbright}')
# plt.rcParams.update(params)

/data/emiln/miniconda2/lib/python2.7/site-packages/pyfits/__init__.py:22: PyFITSDeprecationWarning: PyFITS is deprecated, please use astropy.io.fits
  PyFITSDeprecationWarning)  # noqa


In [2]:
cat_file = '/data/emiln/XLSSC122_GalPops/Data/Products/HST/catalogs/xlssc122_F105_F140_redshifts_short_v4_snr_em_member2_by_hand.cat'
df = pd.read_csv(cat_file,delim_whitespace=True)

In [3]:
df.columns.values

array(['phot-id', 'ra', 'dec', 'f105_kron', 'f105_ekron', 'f105_0p8',
       'f105_e0p8', 'f140_kron', 'f140_ekron', 'f140_0p8', 'f140_e0p8',
       'asec_fwhm', 'zz', 'zlo', 'zhi', 'chi', 'spec-id', 'mcode', 'snr',
       'em-code', 'member1', 'member2'], dtype=object)

In [13]:
df['gold_cluster'] = 0
df['gold_foreground'] = 0
df['silver_cluster'] = 0
df.columns.values
df['(f140)kron'] = df['f140_kron']
for i,r in df.iterrows():
    if (r['member1']>0.5) and (r['(f140)kron']<24 or (r['(f140)kron']<24.5 and r['em-code']==1)):
#         print 'cluster mem'
        df.at[i,'gold_cluster'] = 1
    if (r['member1']<0.5 and r['member1']>0.1) and (r['(f140)kron']<24 or (r['(f140)kron']<24.5 and r['em-code']==1)):
#         print 'cluster mem'
        df.at[i,'silver_cluster'] = 1
    if (r['member2']>0.5) and (r['(f140)kron']<24 or (r['(f140)kron']<24.5 and r['em-code']==1)):
        df.at[i,'gold_foreground'] = 1  
print df['silver_cluster'].sum()
print df['gold_cluster'].sum()
print df['gold_foreground'].sum()
df['is_member'] = df.apply(lambda x: True if x.gold_cluster or x.silver_cluster else False, axis=1)

4
33
13


In [20]:
pd.read_csv('/data/emiln/XLSSC122_GalPops/Data/Products/HST/catalogs/spec_phot_v4_processed.cat')

,phot-id,ra,dec,f105_kron,f105_ekron,f105_0p8,f105_e0p8,f140_kron,f140_ekron,f140_0p8,...,mcode,snr,em-code,member1,member2,gold_cluster,gold_foreground,silver_cluster,(f140)kron,is_member
0,669,34.418431,-3.752946,99.0000,99.0000,99.0000,99.0000,16.5010,0.0014,16.6455,...,1,341.805000,0,0.000000e+00,0.000000e+00,0,0,0,16.5010,False
1,639,34.447945,-3.757976,19.5674,0.0037,19.6684,0.0038,19.4183,0.0052,19.5249,...,1,192.419000,0,0.000000e+00,0.000000e+00,0,0,0,19.4183,False
2,144,34.449888,-3.770144,22.3235,99.0000,99.0000,99.0000,18.7288,0.0047,19.4588,...,1,190.694000,0,0.000000e+00,0.000000e+00,0,0,0,18.7288,False
3,1306,34.436443,-3.739387,20.1216,0.0050,20.2126,0.0052,19.8566,0.0098,19.9529,...,1,153.516000,0,0.000000e+00,0.000000e+00,0,0,0,19.8566,False
4,1128,34.426333,-3.746769,19.8691,0.0043,19.9999,0.0045,19.7036,0.0060,19.8412,...,1,147.213000,0,0.000000e+00,0.000000e+00,0,0,0,19.7036,False
5,439,34.445383,-3.763075,20.3163,0.0055,20.4001,0.0056,20.1515,0.0073,20.2363,...,1,131.656000,0,0.000000e+00,0.000000e+00,0,0,0,20.1515,False
6,280,34.433709,-3.767768,20.5052,0.0057,20.5800,0.0058,20.3568,0.0081,20.4386,...,1,117.368000,0,0.000000e+00,0.000000e+00,0,0,0,20.3568,False
7,595,34.423811,-3.756163,20.6206,0.0070,21.2372,0.0082,20.2988,0.0080,20.9091,...,1,86.035800,0,0.000000e+00,0.000000e+00,0,0,0,20.2988,False
8,209,34.452048,-3.767600,99.0000,99.0000,99.0000,99.0000,19.2985,0.0075,20.4788,...,1,83.911000,0,0.000000e+00,0.000000e+00,0,0,0,19.2985,False
9,1098,34.445601,-3.745351,20.1825,0.0061,21.6684,0.0107,19.9481,0.0073,21.4372,...,1,80.668100,0,0.000000e+00,0.000000e+00,0,0,0,19.9481,False
